# Gestión Moderna de Portafolio
### Autores Bernardo León y Carlos Zapata
### (C) Copyright 2023


## Capitulo 10: Portafolio internacional

In [1]:
#Librerías usadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
%pip install --quiet yfinance
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 6.4 MB/s eta 0:00:00


### Ejemplo 10.1: Modelo Semivarianza
##### Retorno sistematico: BR + X
##### Varianza sistematica: V(BR+X)

In [ ]:
# Información histórica para los mercados locales (DJI,N100,N225) y 
# las tasas de cambio (USDEUR=X, USDJPY=X)
fechai = "2015-12-01"
fechaf = "2020-12-31"
periodicidad = "1Mo"
mercados = ['^DJI','^N100','^N225']
precios = yf.download(mercados,start=fechai,end=fechaf,interval=periodicidad)['Adj Close'].dropna()
retornos = np.log(precios/precios.shift(1)).dropna()
tasas_c = ['USDEUR=X','USDJPY=X']
p_tasas = yf.download(tasas_c,start=fechai,end=fechaf,interval=periodicidad)['Adj Close'].dropna()
r_tasas = np.log(p_tasas/p_tasas.shift(1)).dropna()
retornosx = retornos.join(r_tasas)
rx = retornosx.mean()*12
omega = retornosx.cov()*12
corr = retornosx.corr()
print(rx), print(omega), print(corr)

[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  2 of 2 completed
^DJI        0.112661
^N100       0.039375
^N225       0.073188
USDEUR=X   -0.023543
USDJPY=X   -0.031066
dtype: float64
              ^DJI     ^N100     ^N225  USDEUR=X  USDJPY=X
^DJI      0.024966  0.020444  0.022179 -0.002274  0.001345
^N100     0.020444  0.024604  0.022407 -0.001109  0.002799
^N225     0.022179  0.022407  0.031721 -0.001245  0.006998
USDEUR=X -0.002274 -0.001109 -0.001245  0.003869  0.002089
USDJPY=X  0.001345  0.002799  0.006998  0.002089  0.006460
              ^DJI     ^N100     ^N225  USDEUR=X  USDJPY=X
^DJI      1.000000  0.824872  0.788136 -0.231426  0.105912
^N100     0.824872  1.000000  0.802061 -0.113677  0.222043
^N225     0.788136  0.802061  1.000000 -0.112421  0.488873
USDEUR=X -0.231426 -0.113677 -0.112421  1.000000  0.417802
USDJPY=X  0.105912  0.222043  0.488873  0.417802  1.000000


(None, None, None)

In [ ]:
# Load data MSCI
msci= pd.read_csv("msci.csv")[['MSCI World']]
msci.index = precios.index
r_msci = np.log(msci/msci.shift(1)).dropna()
rm = r_msci.mean()*12
sigmam = r_msci.std()*np.sqrt(12)
round(rm,4), round(sigmam,4)

(MSCI World    0.0962
 dtype: float64, MSCI World    0.1525
 dtype: float64)

In [ ]:
# Estimación de betas - mercados locales
ret = retornos[['^DJI','^N100','^N225']]
n = 3
betas = np.zeros((n,1))
for i in range(n):
    modelo = modelo = sm.OLS(ret.iloc[:,i], sm.add_constant(r_msci)).fit()
    betas[i] = modelo.params[1]
print(betas)

[[0.99609352]
 [0.88692879]
 [0.93422684]]


In [ ]:
diagbetas = np.diagflat(betas)

In [ ]:
# Retorno y riesgo sistemático del portafolio
# Calculos para el portafolio inicial

w = [0.4,0.3,0.3]
f1 = [0,1,0]
f2 = [0,0,1]
B =  np.column_stack([diagbetas, f1, f2]).T
Bw = B @ w
rsp = Bw @ rx
sigmasp = np.sqrt(Bw.T@omega@Bw)
print('E(R+X): ', round(rsp,4)), print('V(R+X): ', round(sigmasp,4))

E(R+X):  0.0595
V(R+X):  0.1521


(None, None)

### Calculos Retorno esperado y Varianza de Markowitz

In [ ]:
# Calculos para el portafolio inicial
w = [0.4,0.3,0.3]
mu = ret.mean()*12
sigma = ret.std()*np.sqrt(12)
datos = pd.concat([mu, sigma], axis=1).T
cov = ret.cov()*12
rp = w @ mu
sigmap = np.sqrt(w @ cov @ w)
print('E(Rp): ', round(rp,4)), print('V(R): ', round(sigmap,4))

E(Rp):  0.0788
V(R):  0.1527


(None, None)

#### Optimización del portafolio

In [ ]:
import cvxpy as cp

In [ ]:
# Optimización Markowitz
n = 3
w = cp.Variable(n)
risk = cp.quad_form(w, cov)
objective = cp.Minimize(risk)
constraints = [cp.sum(w) == 1, w >= 0]
prob = cp.Problem(objective, constraints)
#solvers = ['SCS']
prob.solve()
wpo = pd.DataFrame(w.value)
round(wpo,4)

,0
0,0.4646
1,0.5032
2,0.0322


In [ ]:
# Optimización Markowitz
n = 3
w = cp.Variable(n)
Bw = B @ w
sist_risk = cp.quad_form(Bw, omega)
objective = cp.Minimize(sist_risk)
constraints = [cp.sum(w) == 1, w >= 0]
prob = cp.Problem(objective, constraints)
#solvers = ['SCS']
prob.solve()
wps = pd.DataFrame(w.value)
round(wps,4)

,0
0,0.3782
1,0.6218
2,-0.0000
